In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf

import numpy as np

tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
!pip install tensorflow_privacy

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

In [ ]:
train, test = tf.keras.datasets.cifar10.load_data()
train_data, train_labels = train
test_data, test_labels = test

train_data = np.array(train_data, dtype=np.float32) / 255
test_data = np.array(test_data, dtype=np.float32) / 255

train_data = train_data.reshape(train_data.shape[0], 32, 32, 3)
test_data = test_data.reshape(test_data.shape[0], 32, 32, 3)

train_labels = np.array(train_labels, dtype=np.int32)
test_labels = np.array(test_labels, dtype=np.int32)

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

## Define and tune learning model hyperparameters
Set learning model hyperparamter values. 


In [ ]:
epochs = 15
batch_size = 80

In [ ]:
l2_target = np.arange(1.6, 2.5, 0.1).tolist()
noise_target = np.arange(1.2, 2.1, 0.1).tolist()
noise_default = [1.1 for _ in range(len(l2_target))]
l2_default = [1.5 for _ in range(len(noise_target))]


l2_norm_clip = l2_target + l2_default
noise_multiplier = noise_default + noise_target
num_microbatches = 80
learning_rate = 0.15

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

## Build the learning model

Define a convolutional neural network as the learning model. 

In [ ]:
test_acc_hist = []
eps_hist = []
rdp_hist = []

for i, (l2_clip, noise) in enumerate(zip(l2_norm_clip, noise_multiplier)):
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='valid',
                            activation='relu',
                            input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_clip,
      noise_multiplier=noise,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)

  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  model.fit(train_data, train_labels,
            epochs=epochs,
            validation_data=(test_data, test_labels),
            batch_size=batch_size)
  
  # CALC EPSILON
  eps, rdp = compute_dp_sgd_privacy.compute_dp_sgd_privacy(
      n=train_data.shape[0], 
      batch_size=batch_size, 
      noise_multiplier=noise, 
      epochs=epochs, 
      delta=1 / train_data.shape[0]
      )
  eps_hist.append(eps), rdp_hist.append(rdp)
  
  # CALC ACCURACY
  score, acc = model.evaluate(test_data, test_labels,
                              batch_size=batch_size)
  test_acc_hist.append(acc)

# MOBILENET
TRAIN MOBILENET HERE

In [ ]:
test_acc_hist = []
eps_hist = []
rdp_hist = []

for i, (l2_clip, noise) in enumerate(zip(l2_norm_clip, noise_multiplier)):
  model = tf.keras.applications.mobilenet.MobileNet(
    input_shape=(32, 32, 3), 
    alpha=1.0, 
    depth_multiplier=1, 
    dropout=1e-3, 
    include_top=True, 
    weights=None, 
    input_tensor=None, 
    pooling=None, 
    classes=10
  )

  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_clip,
      noise_multiplier=noise,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)

  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  model.fit(train_data, train_labels,
            epochs=epochs,
            validation_data=(test_data, test_labels),
            batch_size=batch_size)
  
  # CALC EPSILON
  eps, rdp = compute_dp_sgd_privacy.compute_dp_sgd_privacy(
      n=train_data.shape[0], 
      batch_size=batch_size, 
      noise_multiplier=noise, 
      epochs=epochs, 
      delta=1 / train_data.shape[0]
      )
  eps_hist.append(eps), rdp_hist.append(rdp)
  
  # CALC ACCURACY
  score, acc = model.evaluate(test_data, test_labels,
                              batch_size=batch_size)
  test_acc_hist.append(acc)

# RESNET

In [ ]:
test_acc_hist = []
eps_hist = []
rdp_hist = []

for i, (l2_clip, noise) in enumerate(zip(l2_norm_clip, noise_multiplier)):
  model = tf.keras.applications.resnet50.ResNet50(
    include_top=True, 
    weights=None, 
    input_tensor=None, 
    input_shape=(32, 32, 3),
    pooling=None, classes=10
    )

  optimizer = DPGradientDescentGaussianOptimizer(
      l2_norm_clip=l2_clip,
      noise_multiplier=noise,
      num_microbatches=num_microbatches,
      learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(
      from_logits=True, reduction=tf.losses.Reduction.NONE)

  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  model.fit(train_data, train_labels,
            epochs=epochs,
            validation_data=(test_data, test_labels),
            batch_size=batch_size)
  
  # CALC EPSILON
  eps, rdp = compute_dp_sgd_privacy.compute_dp_sgd_privacy(
      n=train_data.shape[0], 
      batch_size=batch_size, 
      noise_multiplier=noise, 
      epochs=epochs, 
      delta=1 / train_data.shape[0]
      )
  eps_hist.append(eps), rdp_hist.append(rdp)
  
  # CALC ACCURACY
  score, acc = model.evaluate(test_data, test_labels,
                              batch_size=batch_size)
  test_acc_hist.append(acc)